In [ ]:
import pickle
import json
import networkx as nx
import datetime
import pandas
import pandas as pd 
from tqdm import tqdm
import numpy as np

# Load Atrial Fibrillation (AF) Patient IDs

In [ ]:
af_patients = pd.read_csv('/projects/ROCKET AF/1A/data extraction/af_ids.csv')
af_ids = list(af_patients['af_ids'])
len(af_ids)

# Step 1: Identifying AF Patients on Warfarin Using Structured Data

In [ ]:
df = pd.read_csv('/projects/data/GS/AF/orders.csv') # Load the orders data from a CSV file
df = df[['client_idcode', 'order_name', 'order_summaryline', 'order_entered', 'order_arrivaldtm']] # Select specific columns
af = df['client_idcode'].isin(af_ids) # Identify AF patients by matching 'client_idcode' with 'af_ids'
df = df[af] # Filter the DataFrame to only keep AF patients' orders
df.shape # Display the shape (number of rows and columns) of the resulting DataFrame

In [ ]:
df.head()

In [ ]:
df['order_name'].unique()

In [ ]:
# Clean and standardize the 'order_name' column to create 'order' column

df['order'] = df['order_name'].str.upper() # Step 1: Create a new 'order' column by converting 'order_name' to uppercase
df['order'] = df['order'].str.replace('[^\w\s]', '', regex=True)  # Step 2: Remove special characters but keep white spaces
df['order'] = df['order'].str.replace(' +', ' ', regex=True) # Step 3: Normalize multiple consecutive spaces to a single space

In [ ]:
df.head()

In [ ]:
# Filter data to only keep patients with 'WARFARIN TABLET' orders

order = ['WARFARIN TABLET']
keep = []
for index, row in df.iterrows():
    keep.append(any([x in row['order'] for x  in order]))
    
df = df.loc[keep] 
df.shape

In [ ]:
df.head()

In [ ]:
# Only keeping the first warfarin prescription order 

df['order_arrivaldtm'] = pd.to_datetime(df['order_arrivaldtm'], utc=True).dt.date 
min_dates = df.groupby('client_idcode')['order_arrivaldtm'].transform('min')
df = df[df['order_arrivaldtm'] == min_dates]
df.shape

In [ ]:
# Removing patients with a mention of those words below in their oder_summaryline

df['order_summaryline_b'] = df['order_summaryline'].str.upper()

matches = ['TWICE','HELD', 'ON HOLD', 'STOPPED', 'STOP', 'WITHHELD', 'WITHHOLD', 'INCREASED', 'RESTARTED', 'INCREASE',
           'DECREASED', 'DECREASE','BD', 'B.D.', 'TDS', 'T.D.S.', 'DISCONTINUED', 'DISCONTINUE']
ids_to_exclude = set()

for i,row in tqdm(df.iterrows()):
    if any(x in df.loc[i,'order_summaryline_b'] for x in matches):
        ids_to_exclude.add(df.loc[i,'client_idcode']) 

In [ ]:
len(ids_to_exclude)

In [ ]:
df = df[~df.client_idcode.isin(ids_to_exclude)]
df.shape

In [ ]:
df.head()

In [ ]:
# Only keeping patients with a mention of 'INR'  in their oder_summaryline

terms = ['INR']
keep = []
for index, row in df.iterrows():
    keep.append(any([x in row['order_summaryline_b'] for x  in terms]))

df = df.loc[keep] 
df.shape    

In [ ]:
df['order_name'].unique()

In [ ]:
# Identify and keep only the first row among duplicates

df = df[~df.duplicated(subset='client_idcode')]
df.reset_index(drop=True, inplace=True)
df.shape

In [ ]:
duplicates = df['client_idcode'].duplicated()
duplicates.value_counts()

In [ ]:
df.head()

In [ ]:
df.iloc[940,2]

In [ ]:
df.iloc[0,2]

In [ ]:
df.iloc[1000,2]

In [ ]:
df.iloc[4000,2]

# Step 2: Identifying AF Patients on Warfarin Using Unstructured Data

In [ ]:
def graph_from_onto(onto):
    G = nx.DiGraph()
    cl_edges = []
    for s, ts in onto.items():
        for t in ts:
            cl_edges.append((s.replace('S-', ''), t.replace('S-', '')))
    G.add_edges_from(cl_edges)
    return G

def load_onto(o):
    with open(f'/projects/data/GS/{o}', 'rb') as f:
        onto = pickle.load(f)
    G = graph_from_onto(onto)
    return G

def expand_codes(onto, codes):
    expanded = {}
    for name, top_codes in codes.items():
        expanded[name] = set()
        for c in top_codes:
            expanded[name].add(c)
            if c in onto:
                expanded[name].update(nx.ancestors(onto, c))
            else:
                print("NOT FOUND:", c, name)
        print(name, len(top_codes), len(expanded[name]))
    return expanded

In [ ]:
onto = load_onto('isa_rela_ch2pt_202009.pickle')

In [ ]:
# entering the SNOMED codes for warfarin

warfarin = {
    'Warfarin' : ['372756006', '777947006']
}

In [ ]:
warfarin = expand_codes(onto,warfarin)

In [ ]:
warfarin_codes = set()
for x in warfarin.values():
    warfarin_codes.update(x)

In [ ]:
%%time 
with open('/projects/data/GS/pt2cui_pos_dates.pickle', 'rb') as f:
    pt2cui_pos_dates = pickle.load(f)

In [ ]:
# This function identifies AF patients with 2 mentions of warfarin

def first_date_for_code_list(pt_data, codes):
    first = None
    for x in codes:
        if x in pt_data and len(pt_data[x])>=2:  
            if first == None:
                first = min(pt_data[x])
            else:
                first = min(first, min(pt_data[x]))
    return first

first_mention_warfarin = {}
for pt in af_ids:
    pt_data = pt2cui_pos_dates.get(pt, {})
    first = first_date_for_code_list(pt_data, warfarin_codes) #
    first_mention_warfarin[pt] = first

In [ ]:
# Filter out the None values (i.e. the non initiators)
    
filtered_first_mention_warfarin = {k: v for k, v in first_mention_warfarin.items() if v is not None}

In [ ]:
df_unstructured_data = pd.DataFrame(filtered_first_mention_warfarin.keys(), columns = ['client_idcode'] )
df_unstructured_data['warfarin_prescription'] = df_unstructured_data['client_idcode'].map(filtered_first_mention_warfarin)
df_unstructured_data['warfarin_prescription'] = pd.to_datetime(df_unstructured_data['warfarin_prescription'], utc=True).dt.date

In [ ]:
df_unstructured_data.shape

In [ ]:
df_unstructured_data.head()

# Step 3: Comparing the structured and unstructured data

In [ ]:
# Keeping patients with no major difference between stuctured and unstrctured warfarin presctiption dates: remove patients if warfarin was mentioned in clinical documents  >=91 days before a warfarin order arrived


def comparison(df1,df2):
    '''
    this function returns 
    :param codes:
    :return data: 
    '''
    df = pd.merge(df1, df2, on='client_idcode', how='outer') 
    df['difference'] =  (pd.to_datetime(df.order_arrivaldtm) - pd.to_datetime(df.warfarin_prescription)).dt.days
    df = df.drop(df[df.difference >= 91].index) 
    df.reset_index(drop=True, inplace=True) 
    # del df ['difference']
    return df

In [ ]:
df = comparison(df,df_unstructured_data)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df['order_name'].isna().sum()

In [ ]:
df = df.dropna(subset=['order_name']) # Removing patients with no prescription records (stuctured data)
df.shape

In [ ]:
warfarin = df[['client_idcode', 'order_arrivaldtm', 'order', 'order_summaryline']]

In [ ]:
warfarin.head()

In [ ]:
first_mention_warfarin = dict(zip(warfarin.client_idcode, warfarin.order_arrivaldtm)) 
len(first_mention_warfarin)

In [ ]:
with open ('/projects/ROCKET AF/1A/warfarin dataextraction/data/first_mention_warfarin.pickle', 'wb') as f:
    pickle.dump(first_mention_warfarin,f)

# Step 1: Identifying AF Patients on Rivaroxaban Using Structured Data

In [ ]:
df = pd.read_csv('/projects/data/GS/AF/orders.csv')
df = df[['client_idcode', 'order_name', 'order_summaryline', 'order_entered', 'order_arrivaldtm']]
af = df['client_idcode'].isin(af_ids)
df = df[af] 
df.shape

In [ ]:
df['order_name'].unique()

In [ ]:
df['order'] = df['order_name'].str.upper()
df['order'] = df['order'].str.replace('[^\w\s]', '', regex=True) 
df['order'] = df['order'].str.replace(' +', ' ', regex=True) 

In [ ]:
%%time
drug_names = ['RIVAROXABAN TABLET']
keep = []
for index, row in df.iterrows():
    keep.append(any([x in row['order'] for x  in drug_names]))

In [ ]:
df = df.loc[keep] 
df.shape

In [ ]:
df['order_arrivaldtm'] = pd.to_datetime(df['order_arrivaldtm'], utc=True).dt.date 
min_dates = df.groupby('client_idcode')['order_arrivaldtm'].transform('min')
df = df[df['order_arrivaldtm'] == min_dates]
df.shape

In [ ]:
df['order_summaryline_b'] = df['order_summaryline'].str.upper()

In [ ]:
# removing patients with a mention of those words below

matches = ['TWICE','HELD', 'ON HOLD', 'STOPPED', 'STOP', 'WITHHELD', 'WITHHOLD', 'INCREASED', 'RESTARTED', 'INCREASE',
           'DECREASED', 'DECREASE','BD', 'B.D.', 'TDS', 'T.D.S.', 'DISCONTINUED', 'DISCONTINUE']
ids_to_exclude = set()

for i,row in tqdm(df.iterrows()):
    if any(x in df.loc[i,'order_summaryline_b'] for x in matches):
        ids_to_exclude.add(df.loc[i,'client_idcode']) 

In [ ]:
len(ids_to_exclude)

In [ ]:
df = df[~df.client_idcode.isin(ids_to_exclude)]
df.shape

In [ ]:
for i,row in tqdm(df.iterrows()):
    if '[ANTICOAGULANT]' not in df.loc[i,'order_summaryline_b']:
        df.loc[i,'dose'] = df.loc[i,'order_summaryline_b'].split('MG')[0].strip()[0:10].strip()
    else:
        df.loc[i,'dose'] = np.nan

for i,row in tqdm(df.iterrows()):
    if '[ANTICOAGULANT]' in df.loc[i,'order_summaryline_b']:
        if '[PRE-PACK]' not in df.loc[i,'order_summaryline_b']: 
            df.loc[i,'dose'] = df.loc[i,'order_summaryline_b'].split('[ANTICOAGULANT]')[1].strip()[0:2].strip()
        else:
            df.loc[i,'dose'] = df.loc[i,'order_summaryline_b'].split('[PRE-PACK]')[1].strip()[0:2].strip()

In [ ]:
df['dose'].unique()

In [ ]:
df.loc[df['dose'] == '5'].head()

In [ ]:
df[df.apply(lambda row: row.astype(str).str.contains('[ANTICOAGULANT]', case=False).any(), axis=1)].head()

In [ ]:
df[df.apply(lambda row: row.astype(str).str.contains('[PRE-PACK]', case=False).any(), axis=1)].head()

In [ ]:
result = df.groupby('client_idcode')['dose'].apply(lambda x: x.nunique() == 1).reset_index(name='same_dose_value')
false_rows = result.loc[result['same_dose_value'] == False]
print(false_rows)

In [ ]:
false_ids = result.loc[result['same_dose_value'] == False, 'client_idcode'].tolist()
len(false_ids)

In [ ]:
df = df[~df['client_idcode'].isin(false_ids)]
df.shape

In [ ]:
df = df.drop_duplicates(subset='client_idcode')
df.reset_index(drop=True, inplace=True)

In [ ]:
df.shape

In [ ]:
df['order_name'].unique()

# Step 2: Indetifying AF patients on Rivaroxaban Using Unstructured data

In [ ]:
# entering the SNOMED codes for rivaroxaban

rivaroxaban = {
    'Rivaroxaban' : ['442031002', '777455008']
}

In [ ]:
rivaroxaban = expand_codes(onto,rivaroxaban)

In [ ]:
rivaroxaban_codes = set()
for x in rivaroxaban.values():
    rivaroxaban_codes.update(x)

In [ ]:
# function to identify AF patients with 2 mentions of rivaroxaban

def first_date_for_code_list(pt_data, codes):
    first = None
    for x in codes:
        if x in pt_data and len(pt_data[x])>=2:  
            if first == None:
                first = min(pt_data[x])
            else:
                first = min(first, min(pt_data[x]))
    return first

first_mention_rivaroxaban = {}
for pt in af_ids:
    pt_data = pt2cui_pos_dates.get(pt, {})
    first = first_date_for_code_list(pt_data, rivaroxaban_codes) #
    first_mention_rivaroxaban[pt] = first

In [ ]:
filtered_first_mention_rivaroxaban = {k: v for k, v in first_mention_rivaroxaban.items() if v is not None}

In [ ]:
df_unstructured_data = pd.DataFrame(filtered_first_mention_rivaroxaban.keys(), columns = ['client_idcode'] )
df_unstructured_data['rivaroxaban_prescription'] = df_unstructured_data['client_idcode'].map(filtered_first_mention_rivaroxaban)
df_unstructured_data['rivaroxaban_prescription'] = pd.to_datetime(df_unstructured_data['rivaroxaban_prescription'], utc=True).dt.date

In [ ]:
df_unstructured_data.head()

In [ ]:
df_unstructured_data.shape

# Step 3: Comparing the structured and unstructured data

In [ ]:
# keeping patients with no major difference between stuctured/unstrctured rivaroxaban presctiptions: if warfarin was mentioned in clinical text 91 days before an order being placed patients are removed


def comparison(df1,df2):
    '''
    this function returns 
    :param codes:
    :return data: 
    '''
    df = pd.merge(df1, df2, on='client_idcode', how='outer') 
    df['difference'] =  (pd.to_datetime(df.order_arrivaldtm) - pd.to_datetime(df.rivaroxaban_prescription)).dt.days
    df = df.drop(df[df.difference >= 91].index) 
    df.reset_index(drop=True, inplace=True) 
    # del df ['difference']
    return df

In [ ]:
df = comparison(df,df_unstructured_data)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df['order_name'].isna().sum()

In [ ]:
df = df.dropna(subset=['order_name']) 
df.shape

In [ ]:
df = df[['client_idcode', 'order_arrivaldtm', 'order_name', 'order_summaryline', 'dose']]

In [ ]:
df.head()

In [ ]:
first_mention_rivaroxaban = dict(zip(df.client_idcode, df.order_arrivaldtm)) 
len(first_mention_rivaroxaban)

In [ ]:
rivaroxaban_dose = dict(zip(df.client_idcode, df.dose)) 
len(rivaroxaban_dose)

In [ ]:
with open ('/projects/ROCKET AF/1A/rivaroxaban dataextraction/data/first_mention_rivaroxaban.pickle', 'wb') as f:
    pickle.dump(first_mention_rivaroxaban,f)

In [ ]:
with open ('/projects/ROCKET AF/1A/rivaroxaban dataextraction/data/rivaroxaban_dose.pickle', 'wb') as f:
    pickle.dump(rivaroxaban_dose,f)